INICIO DO CODIGO
.inicialização
.firmar conexão

In [1]:
#fazer os imports

import panel as pn
import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

#carregar variaveis de ambiente
load_dotenv()

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

#conectar ao banco
con = pg.connect(host=DB_HOST, dbname = DB_NAME, user = DB_USER, password = DB_PASS)
cursor_pg = con.cursor()

#configurar string de conexão
url = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'

cnx = create_engine(url)

TESTE DE CONEXÃO
.acessando o banco de dados

In [2]:
#teste de consulta sql e carregar dataframe do panda
query = "select * from pessoa"
df = pd.read_sql_query(query, cnx)

df

,id_pessoa,nome,sobrenome,email,telefone,senha_hash,data_cadastro
0,1,Ana,Silva,ana.silva@email.com,11987654321,hash123,2024-01-15
1,2,Bruno,Souza,bruno.souza@email.com,11987654322,hash124,2024-01-16
2,3,Carla,Santos,carla.santos@email.com,11987654323,hash125,2024-01-17
3,4,Daniel,Oliveira,daniel.o@email.com,11987654324,hash126,2024-01-18
4,5,Eduarda,Costa,eduarda.c@email.com,11987654325,hash127,2024-01-19
5,6,Felipe,Pereira,felipe.p@email.com,11987654326,hash128,2024-01-20
6,7,Giovana,Almeida,giovana.a@email.com,11987654327,hash129,2024-01-21
7,8,Henrique,Lima,henrique.l@email.com,11987654328,hash130,2024-01-22
8,9,Isabela,Rocha,isabela.r@email.com,11987654329,hash131,2024-01-23
9,10,Joao,Martins,joao.m@email.com,11987654330,hash132,2024-01-24


INTERAÇÕES COM O BANCO DE DADOS
.mostrar/consultar lista de grupos
.mostrar/consultar lista de atendimentos
.inscrever e desinscrever de encontros
.marcar e desmarcar atendimentos
.mostrar participantes de encontros

In [3]:
# Interação com o banco de dados

def get_encontros_df():
    if cnx is None:
        return pd.DataFrame([{"Erro": "Conexão com o banco de dados não estabelecida."}])
    try:
        query = "SELECT id_encontro, titulo, descricao, tipo_encontro, data_hora, link, id_psicologo FROM Encontro ORDER BY data_hora;"
        df = pd.read_sql(query, cnx)
        return df
    except Exception as e:
        return pd.DataFrame([{"Erro": f"Falha ao buscar encontros: {e}"}])

def inscrever_desinscrever_encontro(id_encontro, id_usuario, acao):
  
    if cnx is None:
        return "Erro: Conexão com o banco de dados não estabelecida."
    try:
        with cnx.connect() as connection:
            if acao == 'inscrever':
                check_query = text("SELECT 1 FROM Participa WHERE id_encontro = :id_encontro AND id_usuario = :id_usuario;")
                result = connection.execute(check_query, {'id_encontro': id_encontro, 'id_usuario': id_usuario}).fetchone()
                if result:
                    return f"Usuário {id_usuario} já está inscrito no encontro {id_encontro}."

                insert_query = text("INSERT INTO Participa (id_encontro, id_usuario) VALUES (:id_encontro, :id_usuario);")
                connection.execute(insert_query, {'id_encontro': id_encontro, 'id_usuario': id_usuario})
                connection.commit()
                return f"Usuário {id_usuario} inscrito no encontro {id_encontro} com sucesso!"
            elif acao == 'desinscrever':
                delete_query = text("DELETE FROM Participa WHERE id_encontro = :id_encontro AND id_usuario = :id_usuario;")
                result = connection.execute(delete_query, {'id_encontro': id_encontro, 'id_usuario': id_usuario})
                connection.commit()
                if result.rowcount > 0:
                    return f"Usuário {id_usuario} desinscrito do encontro {id_encontro} com sucesso!"
                else:
                    return f"Usuário {id_usuario} não encontrado como inscrito no encontro {id_encontro}."
            else:
                return "Ação inválida. Use 'inscrever' ou 'desinscrever'."
    except Exception as e:
        return f"Erro ao {acao} usuário do encontro: {e}"

def get_participantes_encontro(id_encontro):
   
    if cnx is None:
        return pd.DataFrame([{"Erro": "Conexão com o banco de dados não estabelecida."}]), "Erro na Conexão"
    
    encontro_titulo = "Encontro Desconhecido"
    try:
        title_query = text("SELECT titulo FROM Encontro WHERE id_encontro = :id_encontro;")
        with cnx.connect() as connection:
            result = connection.execute(title_query, {'id_encontro': id_encontro}).fetchone()
            if result:
                encontro_titulo = result.titulo
            else:
                return pd.DataFrame([{"Info": "Encontro não encontrado."}]), "Encontro não encontrado."

        participants_query = text("""
            SELECT p.id_pessoa, p.nome, p.sobrenome, p.email
            FROM Pessoa p
            JOIN Usuario u ON p.id_pessoa = u.id_usuario
            JOIN Participa pa ON u.id_usuario = pa.id_usuario
            WHERE pa.id_encontro = :id_encontro;
        """)
        df = pd.read_sql(participants_query, cnx, params={'id_encontro': id_encontro})
        
        if df.empty:
            return pd.DataFrame([{"Info": "Nenhum participante encontrado para este encontro."}]), encontro_titulo
        return df, encontro_titulo
    except Exception as e:
        return pd.DataFrame([{"Erro": f"Falha ao buscar participantes: {e}"}]), "Erro na Busca"

def get_encontros_usuario(id_usuario):
    if cnx is None:
        return pd.DataFrame([{"Erro": "Conexão com o banco de dados não estabelecida."}])
    try:
        query = text("""
            SELECT e.id_encontro, e.titulo, e.data_hora, e.tipo_encontro
            FROM Encontro e
            JOIN Participa pa ON e.id_encontro = pa.id_encontro
            WHERE pa.id_usuario = :id_usuario
            ORDER BY e.data_hora;
        """)
        df = pd.read_sql(query, cnx, params={'id_usuario': id_usuario})
        if df.empty:
            return pd.DataFrame([{"Info": "Nenhum encontro encontrado para este usuário ou ID inválido."}])
        return df
    except Exception as e:
        return pd.DataFrame([{"Erro": f"Falha ao buscar encontros do usuário: {e}"}])

def get_atendimentos_df():
    if cnx is None:
        return pd.DataFrame([{"Erro": "Conexão com o banco de dados não estabelecida."}])
    try:
        query = """
            SELECT
                a.id_atendimento,
                p_psicologo.nome AS nome_psicologo,
                p_usuario.nome AS nome_usuario,
                a.data_hora_inicio,
                a.data_hora_fim,
                a.status,
                a.observacoes
            FROM Atendimento a
            JOIN Psicologo ps ON a.id_psicologo = ps.id_psicologo
            JOIN Pessoa p_psicologo ON ps.id_psicologo = p_psicologo.id_pessoa
            JOIN Usuario u ON a.id_usuario = u.id_usuario
            JOIN Pessoa p_usuario ON u.id_usuario = p_usuario.id_pessoa
            ORDER BY a.data_hora_inicio DESC;
        """
        df = pd.read_sql(query, cnx)
        return df
    except Exception as e:
        return pd.DataFrame([{"Erro": f"Falha ao buscar atendimentos: {e}"}])

def search_atendimentos(id_atendimento=None, id_psicologo=None):
  
    if cnx is None:
        return pd.DataFrame([{"Erro": "Conexão com o banco de dados não estabelecida."}])
    
    query_params = {}
    where_clauses = []

    if id_atendimento is not None:
        where_clauses.append("a.id_atendimento = :id_atendimento")
        query_params['id_atendimento'] = id_atendimento
    if id_psicologo is not None:
        where_clauses.append("a.id_psicologo = :id_psicologo")
        query_params['id_psicologo'] = id_psicologo

    where_sql = "WHERE " + " AND ".join(where_clauses) if where_clauses else ""

    query = f"""
        SELECT
            a.id_atendimento,
            p_psicologo.nome AS nome_psicologo,
            p_usuario.nome AS nome_usuario,
            a.data_hora_inicio,
            a.data_hora_fim,
            a.status,
            a.observacoes
        FROM Atendimento a
        JOIN Psicologo ps ON a.id_psicologo = ps.id_psicologo
        JOIN Pessoa p_psicologo ON ps.id_psicologo = p_psicologo.id_pessoa
        JOIN Usuario u ON a.id_usuario = u.id_usuario
        JOIN Pessoa p_usuario ON u.id_usuario = p_usuario.id_pessoa
        {where_sql}
        ORDER BY a.data_hora_inicio DESC;
    """
    try:
        df = pd.read_sql(text(query), cnx, params=query_params)
        if df.empty:
            return pd.DataFrame([{"Info": "Nenhum atendimento encontrado com os critérios fornecidos."}])
        return df
    except Exception as e:
        return pd.DataFrame([{"Erro": f"Falha ao pesquisar atendimentos: {e}"}])

def create_atendimento(id_psicologo, id_usuario, data_hora_inicio, data_hora_fim=None, status="Agendado", observacoes=None):
 
    if cnx is None:
        return "Erro: Conexão com o banco de dados não estabelecida."
    try:
        with cnx.connect() as connection:
            insert_query = text("""
                INSERT INTO Atendimento (id_psicologo, id_usuario, data_hora_inicio, data_hora_fim, status, observacoes)
                VALUES (:id_psicologo, :id_usuario, :data_hora_inicio, :data_hora_fim, :status, :observacoes);
            """)
            connection.execute(insert_query, {
                'id_psicologo': id_psicologo,
                'id_usuario': id_usuario,
                'data_hora_inicio': data_hora_inicio,
                'data_hora_fim': data_hora_fim,
                'status': status,
                'observacoes': observacoes
            })
            connection.commit()
            return "Atendimento criado com sucesso!"
    except Exception as e:
        return f"Erro ao criar atendimento: {e}"

def desmarcar_atendimento(id_atendimento):
   
    if cnx is None:
        return "Erro: Conexão com o banco de dados não estabelecida."
    try:
        with cnx.connect() as connection:
            delete_query = text("DELETE FROM Atendimento WHERE id_atendimento = :id_atendimento;")
            result = connection.execute(delete_query, {'id_atendimento': id_atendimento})
            connection.commit()
            if result.rowcount > 0:
                return f"Atendimento {id_atendimento} desmarcado com sucesso!"
            else:
                return f"Atendimento {id_atendimento} não encontrado."
    except Exception as e:
        return f"Erro ao desmarcar atendimento: {e}"


INICIAR AS EXTENSOES PANEL

In [4]:
#iniciar as extensoes de panel
pn.extension()
pn.extension('tabulador')
pn.extension(notifications = True)

/tmp/ipykernel_14140/1719722831.py:2: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension()


/tmp/ipykernel_14140/1719722831.py:3: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension('tabulador')


/tmp/ipykernel_14140/1719722831.py:4: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension(notifications = True)


CRIAÇÃO DOS COMPONENTES DAS AREAS DE ACESSO
.criação dos widgets para area de usuario/psicologo


In [5]:
# Componentes e widgets da interface do usuario

request_consult_button = pn.widgets.Button(name='Solicitar Consulta', button_type='primary', width=200)
consult_confirmation_message = pn.pane.Markdown(
    "",
    visible=False,
    sizing_mode='stretch_width'
)

show_meetings_button = pn.widgets.Button(name='Mostrar Todos os Encontros', button_type='default', width=250)
meetings_table_pane = pn.pane.DataFrame(pd.DataFrame(), sizing_mode='stretch_width', height=200)

id_encontro_insc_input = pn.widgets.TextInput(name='ID do Encontro (Inscrever/Desinscrever):', placeholder='Ex: 1', width=300)
id_usuario_insc_input = pn.widgets.TextInput(name='Seu ID de Usuário (Inscrever/Desinscrever):', placeholder='Ex: 1', width=300)

insc_button = pn.widgets.Button(name='Inscrever-se', button_type='success', width=150)
desc_button = pn.widgets.Button(name='Desinscrever-se', button_type='warning', width=150)
insc_desc_message = pn.pane.Markdown("", visible=False)

view_participants_input = pn.widgets.TextInput(name='ID do Encontro (Ver Participantes):', placeholder='Ex: 1', width=300)
view_participants_button = pn.widgets.Button(name='Ver Participantes', button_type='light', width=200)
encontro_participants_title_pane = pn.pane.Markdown("", visible=False) # Para exibir o nome do encontro
participants_table_pane = pn.pane.DataFrame(pd.DataFrame(), sizing_mode='stretch_width', height=150)

view_user_meetings_input = pn.widgets.TextInput(name='Seu ID de Usuário (Ver Meus Encontros):', placeholder='Ex: 1', width=300)
view_user_meetings_button = pn.widgets.Button(name='Ver Meus Encontros', button_type='light', width=200)
user_meetings_table_pane = pn.pane.DataFrame(pd.DataFrame(), sizing_mode='stretch_width', height=150)


# Componentes e widgets da interface do psicologo

show_atendimentos_button = pn.widgets.Button(name='Mostrar Todos os Atendimentos', button_type='default', width=300)
atendimentos_table_pane = pn.pane.DataFrame(pd.DataFrame(), sizing_mode='stretch_width', height=250)

search_atendimento_id_input = pn.widgets.TextInput(name='Pesquisar por ID do Atendimento:', placeholder='Ex: 101', width=250)
search_psicologo_id_input = pn.widgets.TextInput(name='Pesquisar por ID do Psicólogo:', placeholder='Ex: 2', width=250)
search_atendimentos_button = pn.widgets.Button(name='Pesquisar Atendimentos', button_type='primary', width=200)
search_atendimentos_message = pn.pane.Markdown("", visible=False)

create_psicologo_id_input = pn.widgets.TextInput(name='ID do Psicólogo:', placeholder='Ex: 2', width=200)
create_usuario_id_input = pn.widgets.TextInput(name='ID do Usuário (Paciente):', placeholder='Ex: 1', width=200)
create_data_hora_input = pn.widgets.TextInput(name='Data e Hora (AAAA-MM-DD HH:MM:SS):', placeholder='Ex: 2025-12-25 10:00:00', width=300)
create_observacoes_input = pn.widgets.TextAreaInput(name='Observações:', placeholder='Breve descrição...', height=80, width=400)
create_atendimento_button = pn.widgets.Button(name='Criar Atendimento', button_type='success', width=200)
create_atendimento_message = pn.pane.Markdown("", visible=False)

desmarcar_atendimento_id_input = pn.widgets.TextInput(name='ID do Atendimento a Desmarcar:', placeholder='Ex: 101', width=300)
desmarcar_atendimento_button = pn.widgets.Button(name='Desmarcar Atendimento', button_type='danger', width=200)
desmarcar_atendimento_message = pn.pane.Markdown("", visible=False)


CALLBACKS
.realizando os callbacks da area de usuario/psicologo

In [6]:
# Callbacks da area do usuario

def on_request_consult_click(event):
    consult_confirmation_message.object = "### Sua consulta foi solicitada, aguarde um psicólogo aceitar."
    consult_confirmation_message.visible = True

def on_show_meetings_click(event):
    encontros_df = get_encontros_df()
    if not encontros_df.empty and "Erro" not in encontros_df.columns:
        meetings_table_pane.object = encontros_df
    else:
        meetings_table_pane.object = pn.pane.Markdown(f"Erro ao carregar encontros: {encontros_df.iloc[0]['Erro'] if 'Erro' in encontros_df.columns else 'Nenhum encontro encontrado.'}")

def on_insc_click(event):
    try:
        e_id = int(id_encontro_insc_input.value)
        u_id = int(id_usuario_insc_input.value)
        message = inscrever_desinscrever_encontro(e_id, u_id, 'inscrever')
        insc_desc_message.object = f"**{message}**"
        insc_desc_message.visible = True
    except ValueError:
        insc_desc_message.object = "**Erro: IDs devem ser números inteiros.**"
        insc_desc_message.visible = True
    except Exception as e:
        insc_desc_message.object = f"**Erro inesperado: {e}**"
        insc_desc_message.visible = True


def on_desc_click(event):
    try:
        e_id = int(id_encontro_insc_input.value)
        u_id = int(id_usuario_insc_input.value)
        message = inscrever_desinscrever_encontro(e_id, u_id, 'desinscrever')
        insc_desc_message.object = f"**{message}**"
        insc_desc_message.visible = True
    except ValueError:
        insc_desc_message.object = "**Erro: IDs devem ser números inteiros.**"
        insc_desc_message.visible = True
    except Exception as e:
        insc_desc_message.object = f"**Erro inesperado: {e}**"
        insc_desc_message.visible = True

def on_view_participants_click(event):
    try:
        e_id = int(view_participants_input.value)
        participants_df, encontro_titulo = get_participantes_encontro(e_id)
        
        if "Encontro não encontrado." in str(encontro_titulo):
            encontro_participants_title_pane.object = f"### Erro: {encontro_titulo}"
            encontro_participants_title_pane.visible = True
            participants_table_pane.object = pd.DataFrame() # Limpa a tabela
        elif "Erro" in participants_df.columns:
            encontro_participants_title_pane.object = f"### Erro ao buscar participantes: {participants_df.iloc[0]['Erro']}"
            encontro_participants_title_pane.visible = True
            participants_table_pane.object = pd.DataFrame() # Limpa a tabela
        else:
            encontro_participants_title_pane.object = f"### Participantes do Encontro: {encontro_titulo}"
            encontro_participants_title_pane.visible = True
            participants_table_pane.object = participants_df

    except ValueError:
        encontro_participants_title_pane.object = "**Erro: O ID do Encontro deve ser um número inteiro.**"
        encontro_participants_title_pane.visible = True
        participants_table_pane.object = pd.DataFrame()
    except Exception as e:
        encontro_participants_title_pane.object = f"**Erro inesperado: {e}**"
        encontro_participants_title_pane.visible = True
        participants_table_pane.object = pd.DataFrame()

def on_view_user_meetings_click(event):
    try:
        u_id = int(view_user_meetings_input.value)
        user_meetings_df = get_encontros_usuario(u_id)
        user_meetings_table_pane.object = user_meetings_df
    except ValueError:
        user_meetings_table_pane.object = pn.pane.Markdown("**Erro: O ID do Usuário deve ser um número inteiro.**")
    except Exception as e:
        user_meetings_table_pane.object = pn.pane.Markdown(f"**Erro inesperado: {e}**")

# Callbacks da area do usuario

def on_show_atendimentos_click(event):
    atendimentos_df = get_atendimentos_df()
    if not atendimentos_df.empty and "Erro" not in atendimentos_df.columns:
        atendimentos_table_pane.object = atendimentos_df
        search_atendimentos_message.visible = False
    else:
        atendimentos_table_pane.object = pd.DataFrame()
        search_atendimentos_message.object = f"**{atendimentos_df.iloc[0]['Erro'] if 'Erro' in atendimentos_df.columns else 'Nenhum atendimento encontrado.'}**"
        search_atendimentos_message.visible = True

def on_search_atendimentos_click(event):
    id_atendimento = None
    id_psicologo = None
    
    try:
        if search_atendimento_id_input.value:
            id_atendimento = int(search_atendimento_id_input.value)
        if search_psicologo_id_input.value:
            id_psicologo = int(search_psicologo_id_input.value)
        
        if id_atendimento is None and id_psicologo is None:
            search_atendimentos_message.object = "**Por favor, insira um ID de Atendimento ou um ID de Psicólogo para pesquisar.**"
            search_atendimentos_message.visible = True
            atendimentos_table_pane.object = pd.DataFrame()
            return

        result_df = search_atendimentos(id_atendimento, id_psicologo)
        
        if not result_df.empty and "Erro" not in result_df.columns:
            atendimentos_table_pane.object = result_df
            search_atendimentos_message.visible = False
        else:
            atendimentos_table_pane.object = pd.DataFrame()
            search_atendimentos_message.object = f"**{result_df.iloc[0]['Erro'] if 'Erro' in result_df.columns else result_df.iloc[0]['Info']}**"
            search_atendimentos_message.visible = True
            
    except ValueError:
        search_atendimentos_message.object = "**Erro: IDs devem ser números inteiros.**"
        search_atendimentos_message.visible = True
        atendimentos_table_pane.object = pd.DataFrame()
    except Exception as e:
        search_atendimentos_message.object = f"**Erro inesperado na pesquisa: {e}**"
        search_atendimentos_message.visible = True
        atendimentos_table_pane.object = pd.DataFrame()

def on_create_atendimento_click(event):
    try:
        p_id = int(create_psicologo_id_input.value)
        u_id = int(create_usuario_id_input.value)
        data_hora_inicio_str = create_data_hora_input.value
        observacoes = create_observacoes_input.value if create_observacoes_input.value else None

        if not data_hora_inicio_str or len(data_hora_inicio_str) < 19: # YYYY-MM-DD HH:MM:SS
            create_atendimento_message.object = "**Erro: Data e Hora de Início inválidas. Use o formato AAAA-MM-DD HH:MM:SS.**"
            create_atendimento_message.visible = True
            return

        message = create_atendimento(p_id, u_id, data_hora_inicio_str, data_hora_fim=None, observacoes=observacoes)
        create_atendimento_message.object = f"**{message}**"
        create_atendimento_message.visible = True
        create_psicologo_id_input.value = ''
        create_usuario_id_input.value = ''
        create_data_hora_input.value = ''
        create_observacoes_input.value = ''
    except ValueError:
        create_atendimento_message.object = "**Erro: IDs devem ser números inteiros. Verifique também o formato da Data e Hora de Início.**"
        create_atendimento_message.visible = True
    except Exception as e:
        create_atendimento_message.object = f"**Erro inesperado ao criar atendimento: {e}**"
        create_atendimento_message.visible = True

def on_desmarcar_atendimento_click(event):
    try:
        a_id = int(desmarcar_atendimento_id_input.value)
        message = desmarcar_atendimento(a_id)
        desmarcar_atendimento_message.object = f"**{message}**"
        desmarcar_atendimento_message.visible = True
        desmarcar_atendimento_id_input.value = ''
    except ValueError:
        desmarcar_atendimento_message.object = "**Erro: O ID do Atendimento deve ser um número inteiro.**"
        desmarcar_atendimento_message.visible = True
    except Exception as e:
        desmarcar_atendimento_message.object = f"**Erro inesperado ao desmarcar atendimento: {e}**"
        desmarcar_atendimento_message.visible = True



DFENININDO LAYOUT DAS AREAS DE ACESSO
.textos iniciais
.botoes
.listas

In [7]:
# Montando as areas de acesso

# Montar a Área do Usuário
user_area_content = pn.Column(
    pn.pane.Markdown("## Bem-vindo à Área do Usuário", styles={'color': '#28a745'}),
    pn.pane.Markdown("Aqui você pode solicitar consultas e gerenciar suas inscrições em encontros.", width=600),
    pn.pane.HTML("<hr>", width=600),

    pn.pane.Markdown("### Solicitar uma Nova Consulta"),
    request_consult_button,
    consult_confirmation_message,

    pn.pane.HTML("<hr>", width=600),

    pn.pane.Markdown("### Encontros Disponíveis"),
    show_meetings_button,
    meetings_table_pane,
    pn.Row(id_encontro_insc_input, id_usuario_insc_input),
    pn.Row(insc_button, desc_button),
    insc_desc_message,

    pn.pane.HTML("<hr>", width=600),

    pn.pane.Markdown("### Visualizar Participantes de um Encontro"),
    pn.Row(view_participants_input, view_participants_button),
    encontro_participants_title_pane,
    participants_table_pane,

    pn.pane.HTML("<hr>", width=600),

    pn.pane.Markdown("### Meus Encontros (Onde Estou Inscrito)"),
    pn.Row(view_user_meetings_input, view_user_meetings_button),
    user_meetings_table_pane,

    sizing_mode='stretch_width'
)

# Montar a Área do Psicólogo
psychologist_area_content = pn.Column(
    pn.pane.Markdown("## Bem-vindo à Área do Psicólogo", styles={'color': '#007bff'}),
    pn.pane.Markdown("Aqui você pode gerenciar seus atendimentos.", width=600),
    pn.pane.HTML("<hr>", width=600),

    pn.pane.Markdown("### Visualizar e Pesquisar Atendimentos"),
    show_atendimentos_button,
    pn.Row(search_atendimento_id_input, search_psicologo_id_input),
    search_atendimentos_button,
    search_atendimentos_message,
    atendimentos_table_pane,

    pn.pane.HTML("<hr>", width=600),

    pn.pane.Markdown("### Criar Novo Atendimento"),
    pn.Row(create_psicologo_id_input, create_usuario_id_input),
    create_data_hora_input,
    create_observacoes_input,
    create_atendimento_button,
    create_atendimento_message,

    pn.pane.HTML("<hr>", width=600),

    pn.pane.Markdown("### Desmarcar Atendimento"),
    pn.Row(desmarcar_atendimento_id_input, desmarcar_atendimento_button),
    desmarcar_atendimento_message,

    sizing_mode='stretch_width'
)


NAVEGAÇÃO DA INICIAL DA PAGINA
.direcionar o usuario a area que deseja

In [8]:
#Navegação principal e events
user_area_content.visible = False
psychologist_area_content.visible = False

main_content_area = pn.Column(sizing_mode='stretch_width')

def show_user_area(event):
    main_content_area.clear()
    main_content_area.append(user_area_content)
    user_area_content.visible = True
    psychologist_area_content.visible = False
    consult_confirmation_message.visible = False
    consult_confirmation_message.object = ""
    insc_desc_message.visible = False
    insc_desc_message.object = ""
    meetings_table_pane.object = pd.DataFrame()
    
    encontro_participants_title_pane.visible = False
    encontro_participants_title_pane.object = ""
    participants_table_pane.object = pd.DataFrame()
    
    user_meetings_table_pane.object = pd.DataFrame()

def show_psychologist_area(event):
    main_content_area.clear()
    main_content_area.append(psychologist_area_content)
    user_area_content.visible = False
    psychologist_area_content.visible = True
    atendimentos_table_pane.object = pd.DataFrame()
    search_atendimentos_message.visible = False
    search_atendimentos_message.object = ""
    create_atendimento_message.visible = False
    create_atendimento_message.object = ""
    desmarcar_atendimento_message.visible = False
    desmarcar_atendimento_message.object = ""
    search_atendimento_id_input.value = ''
    search_psicologo_id_input.value = ''
    create_psicologo_id_input.value = ''
    create_usuario_id_input.value = ''
    create_data_hora_input.value = ''
    create_observacoes_input.value = ''
    desmarcar_atendimento_id_input.value = ''

user_button = pn.widgets.Button(name='Área do Usuário', button_type='primary')
psychologist_button = pn.widgets.Button(name='Área do Psicólogo', button_type='success')

user_button.on_click(show_user_area)
psychologist_button.on_click(show_psychologist_area)

request_consult_button.on_click(on_request_consult_click)
show_meetings_button.on_click(on_show_meetings_click)
insc_button.on_click(on_insc_click)
desc_button.on_click(on_desc_click)
view_participants_button.on_click(on_view_participants_click)
view_user_meetings_button.on_click(on_view_user_meetings_click)

show_atendimentos_button.on_click(on_show_atendimentos_click)
search_atendimentos_button.on_click(on_search_atendimentos_click)
create_atendimento_button.on_click(on_create_atendimento_click)
desmarcar_atendimento_button.on_click(on_desmarcar_atendimento_click)



initial_screen = pn.Column(
    pn.pane.Markdown("# Bem-vindo ao Juntos+", styles={'text-align': 'center', 'font-size': '2em'}),
    pn.pane.Markdown("Por favor, selecione sua área:", styles={'text-align': 'center'}),
    pn.Row(user_button, psychologist_button, align='center', sizing_mode='stretch_width'),
    pn.pane.HTML("<hr>", width=800),
    main_content_area,
    sizing_mode='stretch_width'
)

In [9]:
# Exibe tudo
initial_screen.servable()

Column(sizing_mode='stretch_width')
    [0] Markdown(str, styles={'text-align': 'center', ...})
    [1] Markdown(str, styles={'text-align': 'center'})
    [2] Row(align='center', sizing_mode='stretch_width')
        [0] Button(button_type='primary', name='Área do Usuário')
        [1] Button(button_type='success', name='Área do Psicólogo')
    [3] HTML(str, width=800)
    [4] Column(sizing_mode='stretch_width')